In [1]:
##loading data

import pandas as pd

df=pd.read_csv("personal_expense_classification.csv")
df.head()
df.columns


ModuleNotFoundError: No module named 'pandas'

In [ ]:
df.drop(['expense_id', 'amount', 'merchant'],axis=1,inplace=True)
df.head()
df.columns
df.shape



(200, 2)

In [ ]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.shape

df.isnull().sum()
df.dropna(inplace=True)
df.shape
# df

(187, 2)

In [ ]:
#encoding categorical data

df.columns
df['category'].unique()
# print(df['category'].value_counts())
# print("cateogory" ,df['category'].unique())

# #giving labels to categories

key={
    'shopping':0, 'technology':1 ,'food':2 ,'entertainment':3 ,'transport':4

}
print(key)
df['category']=df['category'].map(key)
df.head()

{'shopping': 0, 'technology': 1, 'food': 2, 'entertainment': 3, 'transport': 4}


,description,category
0,wireless headphones purchase,0
1,monthly streaming subscription,3
2,morning coffee and muffin,2
3,ride to downtown office,4
4,smartphone screen protector,1


In [ ]:
##now preprocess description column
df['description'].unique()

from sklearn.feature_extraction.text import CountVectorizer

X = df['description']       # independent variable
y = df['category']          # target variable

print(y.unique())
print(y.dtype)


vectorizer = CountVectorizer()  #object of CountVectorizer
X_vect = vectorizer.fit_transform(X)  #fit and transform the data

# Check shape
X_vect.shape
  # (num_samples, num_features)

print()
print(" ********************************************************    ")
# Converts text data (description) into numeric vectors so ML models can use it.

# How it works:

# Builds a vocabulary of all words in your dataset

# For each description, counts how many times each word appears

# Example:

# Vocabulary: ['coffee', 'electricity', 'hut', 'pizza', 'starbucks', 'uber']

# Description: "Starbucks coffee" → vector [1, 0, 0, 0, 1, 0]


#IN ABOVE CASE VOCABULARY 

#PRINTING DESC OF FIRST 10 ROWS DESCRIPTION
print(df['description'][:10])

print(end='\n')

# Print the vocabulary
vocab = vectorizer.get_feature_names_out()
print() 
print("Vocabulary (first 20 words):")
print(vocab[:10])  # just show first 20 words for readability
# Print the vector for the first description
vector = X_vect.toarray()[0]
print()

# ?first descrption ?
print("first desciption is  : " ,df['description'][0])
print("\nVector for first description:")
print(vector[:10])  # just show first 20 elements for readability




[0 3 2 4 1]
int64

 ********************************************************    
0      wireless headphones purchase
1    monthly streaming subscription
2         morning coffee and muffin
3           ride to downtown office
4       smartphone screen protector
5                  lunch combo meal
6                    daily bus pass
7        premium music subscription
8       household cleaning supplies
9              iPad protective case
Name: description, dtype: object


Vocabulary (first 20 words):
['accessories' 'ad' 'airport' 'and' 'anime' 'app' 'appliances'
 'appointment' 'art' 'arthouse']

first desciption is  :  wireless headphones purchase

Vector for first description:
[0 0 0 0 0 0 0 0 0 0]


In [ ]:
# training and testing of data

#data is splitted

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)


In [ ]:
#train naive baise model for predicting cateorgy from desc

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))



Accuracy: 0.9473684210526315


In [ ]:
#savin my files model

import joblib

joblib.dump(model,'category_model.joblib')
joblib.dump(vectorizer,'vectorizer_d')

# What You've Successfully Created:
# 1. category_model.joblib - The Brain 🧠

# Contains the trained Naive Bayes model
# Knows the patterns: "coffee" → food, "uber" → transport, etc.
# This is what makes the predictions

# 2. vectorizer_d - The Translator 🔤➡️🔢

# Converts text descriptions into numerical arrays
# CRITICAL: Must use the same vectorizer for new predictions
# Without this, your model won't understand new text


# # ❌ WRONG - This won't work!
# new_text = "Starbucks coffee"
# prediction = model.predict(new_text)  # ERROR! Model can't understand text

# # ✅ RIGHT - This works!
# new_text = "Starbucks coffee"
# vectorized_text = vectorizer.transform([new_text])  # Convert to numbers
# prediction = model.predict(vectorized_text)  # Now model understands!

['vectorizer_d']

In [ ]:
# loading model  and testing 
from sklearn.metrics import accuracy_score

new_expenses = [
    "Uber ride to office",
    "Starbucks coffee",
    " uber new office",
    "Bought new headphones"
]

X_new = vectorizer.transform(new_expenses)


model_l =joblib.load('category_model.joblib')

y_pred_new = model_l.predict(X_new)
# print("Accuracy:", accuracy_score(y_pred, y_pred_new))


category_map = {0:'shopping', 1:'technology', 2:'food', 3:'entertainment', 4:'transport'}

for desc, cat in zip(new_expenses, y_pred_new):
    print(f"Expense: '{desc}' → Predicted Category: {category_map[cat]}")





Expense: 'Uber ride to office' → Predicted Category: transport
Expense: 'Starbucks coffee' → Predicted Category: food
Expense: ' uber new office' → Predicted Category: transport
Expense: 'Bought new headphones' → Predicted Category: technology
